## So sánh

In [84]:
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import torch
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

model = SentenceTransformer("vinai/phobert-base", device='cuda' if torch.cuda.is_available() else 'cpu')

No sentence-transformers model found with name vinai/phobert-base. Creating a new one with mean pooling.


In [64]:
sentence_pairs = [
    ("Tôi thích uống cà phê vào buổi sáng.", "Buổi sáng, tôi hay nhâm nhi ly coffee."),
    ("Trời hôm nay thật đẹp và mát mẻ.", "Thời tiết hôm nay dễ chịu và trong lành."),
    ("Anh ấy đã rời công ty từ tháng trước.", "Tháng rồi, anh ta đã nghỉ việc ở chỗ làm."),
    ("Tôi cảm thấy rất buồn vì kết quả này.", "Kết quả này khiến tôi thất vọng và chán nản."),
    ("Cô ấy là một giáo viên tận tâm và nhiệt huyết.", "Cô ta luôn hết lòng với nghề dạy học.")
]

In [70]:

def get_phobert_embedding(text):
    embedding = model.encode(text)
    return embedding

def cosine_sim_phobert(text1, text2):
    vec1 = get_phobert_embedding(text1).reshape(1, -1)
    vec2 = get_phobert_embedding(text2).reshape(1, -1)
    return cosine_similarity(vec1, vec2)

In [86]:

all_sentences = [s for pair in sentence_pairs for s in pair] 
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences)

# Thêm CountVectorizer
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(all_sentences)

results = []
for idx, (s1, s2) in enumerate(sentence_pairs):
    vec1_tfidf = tfidf_vectorizer.transform([s1])
    vec2_tfidf = tfidf_vectorizer.transform([s2])

    vec1_count = count_vectorizer.transform([s1])
    vec2_count = count_vectorizer.transform([s2])

    cos_tfidf = cosine_similarity(vec1_tfidf, vec2_tfidf)[0][0]
    cos_count = cosine_similarity(vec1_count, vec2_count)[0][0]
    cos_phobert = cosine_sim_phobert(s1, s2)[0][0]

    results.append({
        "Câu 1": s1,
        "Câu 2": s2,
        "CountVectorizer": round(cos_count, 4),
        "TF-IDF": round(cos_tfidf, 4),
        "PhoBERT": round(cos_phobert, 4)
    })

df = pd.DataFrame(results)
df

,Câu 1,Câu 2,CountVectorizer,TF-IDF,PhoBERT
0,Tôi thích uống cà phê vào buổi sáng.,"Buổi sáng, tôi hay nhâm nhi ly coffee.",0.3750,0.2735,0.6888
1,Trời hôm nay thật đẹp và mát mẻ.,Thời tiết hôm nay dễ chịu và trong lành.,0.3536,0.2556,0.6814
2,Anh ấy đã rời công ty từ tháng trước.,"Tháng rồi, anh ta đã nghỉ việc ở chỗ làm.",0.3333,0.2748,0.7316
3,Tôi cảm thấy rất buồn vì kết quả này.,Kết quả này khiến tôi thất vọng và chán nản.,0.4216,0.3331,0.6787
4,Cô ấy là một giáo viên tận tâm và nhiệt huyết.,Cô ta luôn hết lòng với nghề dạy học.,0.1005,0.0791,0.6439


## Demo convert textto token

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("vinai/phobert-base")

embedding = model.encode('Nhóm mình đang tìm hiểu phoBERT', normalize_embeddings=True)
embedding.shape # => So chieu bằng (768,)
embedding

No sentence-transformers model found with name vinai/phobert-base. Creating a new one with mean pooling.


array([ 2.62046494e-02,  4.44544740e-02, -4.97189835e-02, -5.21226749e-02,
       -2.89018191e-02, -1.39739197e-02, -1.23717831e-02, -6.37577102e-02,
       -5.47231035e-03,  2.40074303e-02, -7.62116257e-03,  4.77079749e-02,
        3.49395797e-02,  3.00081279e-02,  3.64663377e-02, -1.40113458e-02,
       -6.90951198e-03, -4.79442859e-03,  1.31978132e-02,  6.15909230e-03,
       -2.48509347e-02, -2.34375857e-02,  5.14799431e-02, -1.32571300e-02,
        1.53941391e-02, -3.84059027e-02,  2.14199107e-02,  2.39090752e-02,
       -3.41699719e-02, -4.79103811e-03, -1.24585489e-02,  6.56613847e-03,
        1.19129810e-02,  4.03618105e-02,  6.44589737e-02,  2.28897426e-02,
        1.10588837e-02,  3.21978591e-02, -3.78268175e-02,  4.04578308e-03,
        2.14840495e-03,  1.23354271e-02,  4.36427817e-02,  2.64563709e-02,
        3.57762761e-02,  1.42547833e-02,  3.22561301e-02, -3.82527001e-02,
        1.17193405e-02,  4.09266651e-02,  5.84150199e-03, -9.08728410e-03,
       -3.86756398e-02,  

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
# Xem ID của từng token
tokens = tokenizer.tokenize("Triển_vọng tươi_sáng , đất nền lập mặt_bằng giá mới Trong số các địa_phương giáp_ranh TP.HCM")
tokens_1 = tokenizer.tokenize("Triển vọng tươi sáng , đất nền lập mặt bằng giá mới Trong số các địa phương giáp ranh TP.HCM")
print(tokens)
print(tokens_1)
  # Ví dụ: 7890


['Triển_vọng', 'tươi_sáng', ',', 'đất', 'nền', 'lập', 'mặt_bằng', 'giá', 'mới', 'Trong', 'số', 'các', 'địa_phương', 'giáp_ranh', 'TP.@@', 'HCM']
['Triển', 'vọng', 'tươi', 'sáng', ',', 'đất', 'nền', 'lập', 'mặt', 'bằng', 'giá', 'mới', 'Trong', 'số', 'các', 'địa', 'phương', 'giáp', 'ranh', 'TP.@@', 'HCM']
